# 2.5. 用插件和智能体增强答疑机器人的能力

## 🚄 前言
在前面的课程中，你已经能够通过优化提示词或者通过优化RAG过程，提高了答疑机器人回复有效、正确信息的能力；但是大模型有一定的局限性，如果你想要答疑机器人帮你搜索公司的报销流程，并让它在钉钉上提交报销申请，即使加入了信息检索知识库等能力，答疑机器人依然无法做到，这是因为大模型无法根据用户的意图，自动调用外部工具来解决问题。

您可以创建智能体（Agent）应用来解决这些问题，以大模型为基础，根据业务场景集成特定外部工具/插件，以此来增强大模型的能力。

## 🍁 课程目标
学完本课程后，你将能够：

* 了解Agent的原理以及运行机制；
* 掌握 Assistant API 的基础编码方法；
* 能够动手使用Assistant API构建Agent应用；


## 📖 课程目录

- [1. 智能体（Agent）](#🤖-1-智能体（agent）)
    - [1.1. 什么是智能体（Agent）](#11-什么是智能体（agent）)
    - [1.2. 运行机制](#13-运行机制)
- [2. 如何构建Agent](#🛠️-2-如何构建agent)
    - [2.1. 输出多模态内容](#21-输出多模态内容)
    - [2.2. 构建你的个性化语音助手](#22-构建你的个性化语音助手)
- [3. 扩展阅读：Multi-Agent](#🤼‍♀️-3-扩展阅读：multi-agent)



## 🤖 1. 智能体（Agent）

### 1.1. 什么是智能体（Agent）
Agent是大模型为基础，能够让大模型自主规划，自动调用外部插件，辅助大模型使用工具与外部世界进行交互的应用。
现有的大模型不仅能通过工具实现与外部世界的交互，也具备规划能力和记忆能力，一个典型的Agent会是这样：

<div align="center">
<img src="https://img.alicdn.com/imgextra/i2/O1CN01pKxr9I1JMv0KOidGH_!!6000000001015-0-tps-1532-694.jpg" alt="image" style="width:800px;height:350px;">
</div>

如果你还想了解智能体的功能及应用场景，请浏览大模型工程师ACA认证课程中-第6节:[借助Agent让大模型应用思考、决策并执行任务](https://edu.aliyun.com/course/3126500/lesson/342570389?spm=a2cwt.28196072.ACA13.11.22036e7bIVcV0V)的相关内容
### 1.2. 运行机制
Agent先接受用户输入，再由内置的大语言模型判断是否需要调用工具。

- 如果需要调用工具，大语言模型会选择合适的工具，工具返回结果和用户内容合并后再次输入到大语言模型，由大语言模型生成内容并输出。

- 如果无需调用工具，大语言模型将直接生成内容并输出。

<div align="center">
<img src="https://img.alicdn.com/imgextra/i4/O1CN01bJe4vz1epryTreCNG_!!6000000003921-0-tps-828-752.jpg" alt="image" style="width:400px;height:400px;">
</div>

## 🛠️ 2. 如何构建Agent
我们将使用Assistant API构建Agent，Assistant API是一款开发工具，旨在帮助您快速构建智能体应用（Assistant）。相比于控制台上的图形化应用，Assistant API 提供更灵活的代码开发方式，便于实现深度业务集成。其内置的会话管理和插件机制也简化了开发流程。你可以在[Assistant API](https://help.aliyun.com/zh/model-studio/developer-reference/assistantapi/?spm=a2c4g.11186623.0.0.1e862bdbrazRM0)手册查看各组件的参数详解。

### 2.1. 输出多模态内容
文本生成模型自身不具备生成图像的功能，一般需要通过特定的文生图模型，可以将文本转化为图像。在这里借助 Assistant API 创建的智能体应用，然后调用官方预置的互联网搜索和文生图工具，来构建一个绘画助手。具体操作步骤如下：

#### 步骤1. 计算环境准备
##### 1.1 安装依赖

In [ ]:
! pip install -r requirements.txt

##### 1.2 导入依赖

In [12]:
import json
import os
import dashscope
from http import HTTPStatus

##### 1.3 设置环境变量

设置DASHSCOPE_API_KEY环境变量

In [ ]:
import sys
sys.path.append("../")
from config.load_key import load_key
load_key()
print(os.environ["DASHSCOPE_API_KEY"])

#### 步骤 2. 封装调用状态查询函数

In [14]:
def check_status(component, operation):
    if component.status_code == HTTPStatus.OK:
        print(f"{operation} 成功。")
        return True
    else:
        print(f"{operation} 失败。状态码：{component.status_code}，错误码：{component.code}，错误信息：{component.message}")
        return False

#### 步骤 3. 创建绘画助手Assistant
我们使用 Assistant 类的 create 方法创建 Assistant 智能体。这一过程涉及以下关键参数的设定：

- model：大语言模型名，用于为智能体配置大模型    
- name：智能体名称，用于区别智能体的名称   
- description：智能体的功能描述    
- instructions：由自然语言构成的指令，用于定义智能体的角色和任务    
- tools：智能体配置的工具列表，其中type表示插件类型，description表示对插件的描述

如果你想使用通义千问的大模型构建自己的智能助手，你只需要在description参数填写你的智能助手的功能，在instructions参数中填写智能体的角色和任务即可。

具体到我们的案例，目标是构建一个专注于绘画的 Assistant。基于文生图工具对语言理解能力的较高要求，我们选用了通义千问-Max作为文本推理模型，以此强化 Assistant 的语义理解和文本生成能力。  

智能体的配置细节，包括其名称、功能描述及指令，均在随附的代码片段中有明确展示。为了丰富智能体的功能性和实用性，我们集成了两个官方预置插件：

- quark_search，夸克搜索，旨在帮助智能体获取关于宠物猫的广泛文本信息；

- text_to_image，图片生成，确保智能体能够根据接收到的文字描述，自动生成相应的图像内容。

In [15]:
# 使用 Qwen-Max 创建一个新的绘画助手
painting_assistant = dashscope.Assistants.create(
    model='qwen-max' ,  # model大语言模型名，用于为智能体配置大模型
    name='Art Maestro', # name智能体名称，用于区别智能体的名称
    description='用于绘画和艺术知识的AI助手', # description智能体的功能描述
    instructions='''提供绘画技巧、艺术史和创意指导的信息。
    使用工具进行研究和生成图像。''', # 由自然语言构成的指令，用于定义智能体的角色和任务
    tools=[
        {'type': 'quark_search', 'description': '用于研究艺术主题'},
        {'type': 'text_to_image', 'description': '用于创建视觉示例'}
    ]
) # tools为智能体配置的工具列表

# 打印助手的ID以确认创建成功
if not check_status(painting_assistant, "助手创建"):
    exit()
print(f"绘画助手 'Art Maestro' 创建成功，ID：{painting_assistant.id}")

助手创建 成功。
绘画助手 'Art Maestro' 创建成功，ID：asst_29e7442b-c0df-4e1a-be99-a964c1136631


#### 步骤 4. 创建Thread
Thread（线程）是 Assistant API 中的一个关键概念，它代表了一个持续的对话上下文。

Thread 允许您在用户启动新对话时创建一个会话管理线程。Assistant 可以通过 Thread 理解整个对话的上下文，从而提供更加连贯和相关的回应。在绘画助手的场景中，Thread 可以跟踪用户的初始请求、Assistant 的初步建议、用户的反馈，以及最终的绘画结果，形成一个完整的创作过程。这确保了整个创作过程的连贯性和可追溯性。

In [16]:
# 创建一个新的空线程
thread = dashscope.Threads.create()

if not check_status(thread, "线程创建"):
    exit()

# 注意：这个空线程现在可以用于保持你绘画项目讨论的上下文，
# 包括任何关于布偶猫或其他绘画主题的未来消息。

线程创建 成功。


#### 步骤 5. 添加 Message 到 Thread

您输入的内容将通过 Message 对象传递。 Assistant API支持向单一 Thread 发送一个或多个消息。创建 Message 时，您需要考虑以下参数：

- Thread 唯一编号thread_id

- 消息内容content

尽管 Thread 可接收的 Token 数量无硬性限制，但实际传递给大模型的 Token 数量需符合该模型的最大输入长度限制，具体参照各通义千问系列模型的官方文档中关于上下文长度的说明。在我们的场景中，您将通过 Message 在 Thread 中发出第一条消息：“请帮我画一只布偶猫的图片”。您需要创建一个 Message 类，详细参数设置均在随附的代码片段中。

In [17]:
# 创建一条消息，告诉助手需要做什么？
message = dashscope.Messages.create(thread.id, content='请帮我画一张宠物猫的漫画。')
if not check_status(message, "消息创建"):
    exit()

消息创建 成功。


####  步骤 6. 创建Run并执行
用户将消息分配至特定 Thread 后，可以通过启动一个运行（Run）来激活预设的助手（Assistant）。该助手会以线程内的所有消息为背景，利用指定的模型和可用的插件，智能地回应用户的问题，并将生成的答案插入到线程的消息序列中。

在本场景中，您需执行以下步骤：

- 初始化一个运行对象，以驱动绘画助手，传递线程ID（thread.id）和助手ID（assistant.id）。

- 使用运行对象的等待方法（Run.wait），直至执行完毕。

In [18]:
run = dashscope.Runs.create(thread.id, assistant_id=painting_assistant.id)

if not check_status(run, "运行创建"):
    exit()

print("等待助手处理请求...")
run = dashscope.Runs.wait(run.id, thread_id=thread.id)

if check_status(run, "运行完成"):
    print(f"运行完成，状态：{run.status}")
else:
    print("运行未完成。")
    exit()

运行创建 成功。
等待助手处理请求...
运行完成 成功。
运行完成，状态：completed


#### 步骤 7. 返回结果
这一步通过消息列表方法（Messages.list），检索助手绘制的宠物猫图片。这一系列操作确保了助手从接收问题到输出结果的自动化处理流程。

In [19]:
import re

messages = dashscope.Messages.list(thread.id)

if check_status(messages, "消息检索"):
    if messages.data:
        # 显示最后一条消息的内容（助手的响应）
        last_message = messages.data[0]
        print("\n助手的回应：")

        # 提取消息内容
        message_content = last_message['content'][0]['text']['value']

        # 使用正则表达式找到图片链接
        pattern = r'!\[\]\((.*?)\)'
        matches = re.findall(pattern, message_content)

        # 输出文本信息和单独的链接
        text_without_links = re.sub(pattern, '', message_content).strip()
        print("助手的文字回应：")
        print(text_without_links)

        # 输出所有找到的链接
        if matches:
            print("\n以下是图片链接，可以直接点击打开：")
            for link in matches:
                print(link)
        else:
            print("\n未找到任何图片链接。")
    else:
        print("在线程中未找到消息。")
else:
    print("未能检索到助手的响应。")

消息检索 成功。

助手的回应：
助手的文字回应：
这是为您创作的宠物猫漫画形象，希望您会喜欢：



这张漫画中的猫咪是不是既可爱又生动呢？如果您有任何其他要求或需要进一步调整，请随时告诉我。

以下是图片链接，可以直接点击打开：
https://dashscope-result-sh.oss-cn-shanghai.aliyuncs.com/1d/f5/20241017/29c02cfd/35642780-3e3a-44d5-acb8-1fdd04fd642c-1.png?Expires=1729241381&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=X%2B3Na010kYu213sRSxnveouaPiQ%3D


### 2.2. 构建你的个性化语音助手

在2.1的案例中，我们使用的是官方内置的工具来构建Agent，该应用只能实现单轮对话，也没有加入私有知识库；下面这个案例将结合私有知识库，编写本地自定义插件来构建多轮对话的Agent。

本案例基于百炼上已经构建的知识库，构建知识库的方法请参考-知识库；并结合本地的语音合成工具，帮助大模型输出文本语言的同时，还能输出音频数据；你可以根据你业务本身的需要，按照下列方法构建自己的工具，来实现更加定制化的应用。流程如下：

<div align="center">
<img src="https://img.alicdn.com/imgextra/i4/O1CN01W1jx4b1sJGOZL0lU1_!!6000000005745-0-tps-1046-786.jpg" alt="image" style="width:500px;height:360px;">
</div>

**前期准备**：

 （1）本案例需要通过获取用户在百炼平台的API-KEY来调用大模型，参考链接：[获取API_KEY](https://help.aliyun.com/zh/model-studio/developer-reference/get-api-key?spm=a2c4g.11186623.0.0.2f252a02dyN9ui)

 （2）本案例需要依赖知识库获取私有信息，并获取知识库ID，参考链接[创建知识库并获取知识库ID](https://help.aliyun.com/zh/model-studio/user-guide/rag-knowledge-base?spm=a2c4g.11186623.0.0.193bfa89ZwRJCv)
  
 （3）本案例需要获取知识库所属的业务空间ID，参考链接：[获取工作空间ID](https://help.aliyun.com/zh/model-studio/developer-reference/use-workspace?spm=a2c4g.11186623.0.0.79a535a2fnDIyz&scm=20140722.S_help@@%E6%96%87%E6%A1%A3@@2587495@@2.S_llmOS0.ID_698593-RL_%E7%99%BE%E7%82%BC%E5%B7%A5%E4%BD%9C%E7%A9%BA%E9%97%B4ID-LOC_chat~DAS~llm-OR_ser-PAR1_212bee0f17287002915655784d0095-V_3-P0_0)


<details>
<summary>(4)本案例需要调用阿里云API，需要获取AccessKey和AccessKeySecret，完成身份验证，点击本标题前面的展开按钮即可查看获取方法</summary>

您可以为阿里云主账号和子账号创建一个访问密钥（AccessKey）。在调用阿里云API时您需要使用AccessKey完成身份验证。AccessKey包括AccessKey ID和AccessKey Secret。
- AccessKeyId：用于标识用户。
- AccessKeySecret：用于验证用户的密钥。AccessKeySecret必须保密。

**获取步骤如下所示：**

1. 以主账号登录[阿里云管理控制台](https://home.console.aliyun.com/new?spm=a2c4g.11186623.0.0.7a341a03V3cFLa#/)。
2. 将鼠标置于页面右上方的账号图标，单击accesskeys。
3. 在安全提示页面，选择获取主账号还是子账号的Accesskey.
<div align="center">
<img src="https://img.alicdn.com/imgextra/i2/O1CN012Ob5P81c47RotxqQX_!!6000000003546-2-tps-736-280.png" alt="image" style="width:400px;height:150px;">
</div>

4. 获取主账号Accesskey
   
- 单击继续使用AccessKey。
- 在安全信息管理页面，单击创建AccessKey。
- 在手机验证页面，获取验证码，完成手机验证，单击确定。
- 在新建用户AccessKey页面，展开AccessKey详情，查看AccessKeyId和AccessKeySecret。可以单击保存AK信息，下载AccessKey信息。
<div align="center">
<img src="https://img.alicdn.com/imgextra/i3/O1CN01Y6JVr91KnlqI4hWLN_!!6000000001209-2-tps-727-505.png" alt="image" style="width:350px;height:200px;">
</div>


5. 获取子账号Accesskey
- 单击开始使用子用户AccessKey。
- 如果未创建RAM用户，在系统跳转的[RAM访问控制台](https://ram.console.aliyun.com/users/new?spm=a2c4g.11186623.0.0.7a341a03V3cFLa)的新建用户页面，创建RAM用户。如果是获取已有RAM用户的Accesskey，则跳过此步骤。
- 在[RAM访问控制台](https://ram.console.aliyun.com/users/new?spm=a2c4g.11186623.0.0.7a341a03V3cFLa)的左侧导航栏，选择人员管理 > 用户，搜索需要获取AccessKey的用户。
- 单击用户登录名称，在用户详情页认证管理页签下的用户AccessKey区域，单击创建新的AccessKey
- 在手机验证页面，获取验证码，完成手机验证，单击确定。
- 在创建AccessKey页面，查看AccessKeyId和AccessKeySecret。可以单击下载CSV文件，下载AccessKey信息或者单击复制，复制AccessKey信息。
<div align="center">
<img src="https://img.alicdn.com/imgextra/i1/O1CN0126XluY1iMnIBxEE8G_!!6000000004399-2-tps-775-428.png" alt="image" style="width:350px;height:200px;">
</div>
</details>

#### 步骤1. 计算环境准备
##### 1.1 安装依赖

In [ ]:
! pip install -r requirements.txt

##### 1.2 导入依赖


In [1]:
import os
import json
import IPython
import getpass

from http import HTTPStatus
from dashscope.audio.tts_v2 import *
from dashscope import Application,Assistants,Messages, Runs, Threads

from alibabacloud_bailian20231229.client import Client as bailian20231229Client
from alibabacloud_tea_openapi import models as open_api_models
from alibabacloud_bailian20231229 import models as bailian_20231229_models
from alibabacloud_tea_util import models as util_models
from alibabacloud_tea_util.client import Client as UtilClient

##### 1.3 设置环境变量

设置DASHSCOPE_API_KEY环境变量:

In [ ]:
import sys
sys.path.append("../")
from config.load_key import load_key
load_key()
print(os.environ["DASHSCOPE_API_KEY"])

设置ALIBABA_CLOUD_ACCESS_KEY_ID、ALIBABA_CLOUD_ACCESS_KEY_SECRET环境变：

In [3]:
os.environ["ALIBABA_CLOUD_ACCESS_KEY_ID"] = getpass.getpass("请输入你的access_key:")

请输入你的access_key: ········


In [4]:
os.environ["ALIBABA_CLOUD_ACCESS_KEY_SECRET"] = getpass.getpass("请输入你的access_secret:")

请输入你的access_secret: ········


#### 步骤 2. 创建工具函数
用户可以自行准备各类待使用的私有API或者Function能力，用于处理本地调用的请求。

In [5]:
def get_audio(message,model='cosyvoice-v1',voice="longxiaochun"):
    synthesizer = SpeechSynthesizer(model=model, voice=voice)
    audio = synthesizer.call(message)
    # print('requestId: ', synthesizer.get_last_request_id())
    with open('out.mp3', 'wb') as f:
        f.write(audio)
    audio_play = IPython.display.Audio(audio)
    display(audio_play)
    return '音频合成成功'

<!-- #### 步骤 3. 调用百炼应用的RAG应用 -->

#### 步骤 3. 调用Retrieve - 检索知识索引API，检索知识库内容
- create_client方法：初始化阿里云账号
- get_retrieve方法：用于知识库检索，返回与用户的问题最相近的知识库片段，详细请见[Retrieve API工具](https://help.aliyun.com/zh/model-studio/developer-reference/api-bailian-2023-12-29-retrieve?spm=a2c4g.11186623.0.0.2219fa89tibuDg)

本案例知识库上传的是《百炼系列手机产品介绍.docx》文档，详见“大模型ACP认证教程/p2_构造大模型问答系统/docs”文件夹。

In [6]:
class Client:
    def __init__(self,query,dense_similarity_top_k,enable_rewrite,enable_reranking,
                 rerank_min_score,rerank_top_n,sparse_similarity_top_k,
                 index_id,save_retriever_history,WorkspaceId):
        self.query = query
        self.dense_similarity_top_k = dense_similarity_top_k
        self.enable_rewrite = enable_rewrite
        self.enable_reranking = enable_reranking
        self.rerank_min_score = rerank_min_score
        self.rerank_top_n = rerank_top_n
        self.sparse_similarity_top_k = sparse_similarity_top_k
        self.index_id = index_id
        self.save_retriever_history = save_retriever_history
        self.WorkspaceId = WorkspaceId

    @staticmethod
    def create_client() -> bailian20231229Client:
        """
        使用AK&SK初始化账号Client
        @return: Client
        @throws Exception
        """
        config = open_api_models.Config(
            # 必填，请确保代码运行环境设置了环境变量 ALIBABA_CLOUD_ACCESS_KEY_ID。,
            access_key_id=os.environ['ALIBABA_CLOUD_ACCESS_KEY_ID'],
            # 必填，请确保代码运行环境设置了环境变量 ALIBABA_CLOUD_ACCESS_KEY_SECRET。,
            access_key_secret=os.environ['ALIBABA_CLOUD_ACCESS_KEY_SECRET']
        )
        # Endpoint 请参考 https://api.aliyun.com/product/bailian
        config.endpoint = f'bailian.cn-beijing.aliyuncs.com'
        return bailian20231229Client(config)

    def get_retrieve(self):
        client = Client.create_client()
        rewrite_0 = bailian_20231229_models.RetrieveRequestRewrite(
            model_name='conv-rewrite-qwen-1.8b'
        )
        rerank_0 = bailian_20231229_models.RetrieveRequestRerank(
            model_name='gte-rerank-hybrid'
        )
        retrieve_request = bailian_20231229_models.RetrieveRequest(
            query=self.query, # 输入文本（原始输入 prompt）
            dense_similarity_top_k=self.dense_similarity_top_k, # 向量检索 Top K，通过生成输入文本的向量并在知识库中检索与其向量表示最相似的 K 个文本切片
            enable_reranking=self.enable_reranking, # 是否开启 Rerank 重排序，默认为true
            enable_rewrite=self.enable_rewrite, # 是否开启多轮会话改，默认为false，它会基于会话上下文自动调整原始输入 prompt（用户问题）以提升检索效果。
            rerank=[
                rerank_0
            ], # Rank 模型名称
            rerank_min_score=self.rerank_min_score, # 相似度阈值。该阈值表示允许召回的文本切片的最低相似度分数
            rerank_top_n=self.rerank_top_n, # Rerank 后的 Top N 返回数据
            rewrite=[
                rewrite_0
            ], # 会话改写模型名称
            sparse_similarity_top_k=self.sparse_similarity_top_k, # 关键词检索 TopK，即在知识库中查找与输入文本的关键词精确匹配的切片
            index_id=self.index_id, # 知识库 ID
            save_retriever_history=self.save_retriever_history, # 是否保存历史文本切片召回测试数据
        )
        runtime = util_models.RuntimeOptions()
        headers = {}
        try:
            # 复制代码运行请自行打印 API 的返回值
            result = client.retrieve_with_options(self.WorkspaceId, retrieve_request, headers, runtime)
            return result
        except Exception as error:
            # 此处仅做打印展示，请谨慎对待异常处理，在工程项目中切勿直接忽略异常。
            # 错误 message
            print(error.message)
            # 诊断地址
            print(error.data.get("Recommend"))
            UtilClient.assert_as_string(error.message)

#### 步骤 4. 基于Assistant API，实现Function调用
实现在通过检索文档的基础上，完成各类本地function调用的能力，本案例是实现文本转音频的输出，对工具的描述如下：
  - name: 文本生成语音，也就是前面封装好的函数get_audio，将在智能体中被调用。
  - description: 提供了该工具的描述，帮助用户理解其用途。
  - parameters: 定义了函数的参数，这里是大模型的输出 message，且这是必传函数

In [7]:
def create_assistant():
    assistant = Assistants.create(
        model="qwen-max",
        name='smart helper',
        description='一个智能语音助手，可以回答用户问题，请你用简洁的语言回答，如果用户提供的信息不全，你需要反问他，让他提供没有提供的参数。',
        instructions='需要将结合用户的输入，并使用语音合成插件将大模型的回答输出音频',
        tools=[
            {
                'type': 'function',
                'function': {
                    'name': '文本生成语音',
                    'description': '将大模型生成的文本合成为音频',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'message': {
                                'type': 'str',
                                'description': '大模型生成的文本，将会被转换为音频'
                            },
                        },
                        'required': ['message']},
                },
            },
        ]
        
    )

    return assistant

- function_mapper：构建插件字典，键表示插件的描述，值为插件对应的函数名
- thread：创建对话线程，Assistant 可以通过 Thread 理解整个对话的上下文，从而提供更加连贯和相关的回应，对话只要传入同一个线程ID，则表示同一组对话，建议您：

    - 为每个新用户或新的对话主题创建一个新的Thread。   
    - 在需要保持上下文的情况下，继续使用同一个Thread。
    - 当对话主题发生显著变化时，考虑创建新的Thread以避免上下文混淆。

In [8]:
function_mapper = {
    "文本生成语音": get_audio
}

# create a thread.
thread = Threads.create()
print(thread)

{'id': 'thread_b82f67ff-fcc5-48e2-bcfe-1c92b692d8d0', 'object': 'thread', 'created_at': 1729166884955, 'metadata': {}, 'request_id': 'a5799882-acfd-95b9-86dd-c234f7081804', 'status_code': 200}


send_message 模块用来传递message给assistant

In [13]:
def send_message(assistant, message=''): 
    print(f"用户的问题: {message}")
    print("="*30)
    
# create a message.
    message = Messages.create(thread.id, content=message)

    run = Runs.create(thread.id, assistant_id=assistant.id,stream=False)

    run_status = Runs.wait(run.id, thread_id=thread.id)

    if run_status.status == 'failed':
        print('run failed:')
        print(run_status.last_error)
    if run_status.required_action:

        f = run_status.required_action.submit_tool_outputs.tool_calls[0].function
        func_name = f['name']
        param = json.loads(f['arguments'])
        print(f"调用的工具为, {f['name']}-{function_mapper[func_name]}, 参数为: {param['message']}")
        if func_name in function_mapper:
            output = function_mapper[func_name](**param)
        else:
            output = ""

        tool_outputs = [{      
            'output':
                output
        }]


        run = Runs.submit_tool_outputs(run.id,
                                       thread_id=thread.id,
                                       tool_outputs=tool_outputs)

        # should wait for run completed
        run_status = Runs.wait(run.id, thread_id=thread.id)
        
    msgs = Messages.list(thread.id)
    # 如果想查看大模型与用户的所有对话结果，请将下列代码取消注释
    # print("="*30)
    # print("大模型的运行结果:")
    # for message in msgs['data'][::-1]:
    #     print("role: ",message["role"])
    #     print("content: ", message['content'][0]['text']['value'])
    print("\n")

- client是初始化之前创建检索知识库内容的类，其中index_id和WorkspaceId是在**前期准备**中获取的百炼平台的知识库ID和工作空间ID
- send_message函数得到大模型的输出
  
**注意：请参考前文介绍前期准备替换下面的信息 index_id='14xxxxxx' 和 WorkspaceId ='llm-xxxxxx'**

In [ ]:
assistant = create_assistant()
history = ''
while True:
    recall_message = ''
    prompt = input("用户请输入：")
    client = Client(query=prompt,
                dense_similarity_top_k=100,
                enable_reranking=True,
                enable_rewrite=True,
                rerank_min_score=0.5,
                rerank_top_n=5,
                sparse_similarity_top_k=100,
                index_id='xxxxxx', # 知识库ID
                save_retriever_history =True,
                WorkspaceId ='llm-xxxxxx') # 工作空间ID
    chunck_list = client.get_retrieve()
    nodes = chunck_list.body.data.nodes
    for node in nodes:
        recall_message += node.text

    print("="*30)
    if recall_message =='':
         print("从知识库召回的内容为空，知识库中没有相关内容")
    else:  
        print("从知识库召回的内容为：",recall_message)
    print("="*30)
    send_message(assistant=assistant, message= "请你参考以下信息，回答用户的问题" + recall_message +'，'+ prompt +'，请必须调用语音合成插件将大模型的每一条回答转换成音频')

用户请输入： 你是谁


从知识库召回的内容为空，知识库中没有相关内容
用户的问题: 请你参考以下信息，回答用户的问题，你是谁，请必须调用语音合成插件将大模型的每一条回答转换成音频
调用的工具为, 文本生成语音-<function get_audio at 0x10949f7e0>, 参数为: 我是您的智能语音助手，可以回答您的问题、提供信息帮助等。有什么我可以帮到您的吗？


用户请输入： 推荐百炼产品中适合玩游戏的手机


从知识库召回的内容为： 参考售价：5999- 6499。百炼 Ace Ultra ——游戏玩家之选：配备 6.67英寸 1080 x 2400像素屏幕，内置 10GB RAM与 256GB存储，确保游戏运行丝滑无阻。百炼 Ace Ultra ——游戏玩家之选：配备 6.67英寸 1080 x 2400像素屏幕，内置 10GB RAM与 256GB存储，确保游戏运行丝滑无阻。5500mAh电池搭配液冷散热系统，长时间游戏也能保持冷静。高动态双扬声器，沉浸式音效升级游戏体验。参考售价：3999- 4299。百炼 Zephyr Z9 ——轻薄便携的艺术：轻巧的 6.4英寸 1080 x 2340像素设计，搭配 128GB存储与 6GB RAM，日常使用游刃有余。4000mAh电池确保一天无忧，30倍数字变焦镜头捕捉远处细节，轻薄而不失强大。参考售价：2499- 2799。百炼 Flex Fold+ ——折叠屏新纪元：集创新与奢华于一身，主屏 7.6英寸 1800 x 2400像素与外屏 4.7英寸 1080 x 2400像素，多角度自由悬停设计，满足不同场景需求。512GB存储、12GB RAM，加之 4700mAh电池与 UTG超薄柔性玻璃，开启折叠屏时代新篇章。此外，这款手机还支持双卡双待、卫星通话，帮助您在世界各地都能畅联通话。参考零售价：9999- 10999。欢迎来到未来科技的前沿，探索我们精心打造的智能手机系列，每一款都是为了满足您对科技生活的无限遐想而生。百炼 X1 ——畅享极致视界：搭载 6.7英寸 1440 x 3200像素超清屏幕，搭配 120Hz刷新率，流畅视觉体验跃然眼前。256GB海量存储空间与 12GB RAM强强联合，无论是大型游戏还是多任务处理，都能轻松应对。5000mAh电池长续航，加上超感光四摄系统，记录生活每一刻精彩。参考售价：4599- 4999通义 Vivid 7 ——智能摄影新体验：拥有 6.5英寸 1080 x 2400像素全面屏，AI智能摄影功能让每一张照片都能展现专业级色彩与细节。8GB RAM与 128GB存储空间确保流畅操作，4500mAh电池满足日常所需。侧面指纹解锁，便捷又安全。参考售价：2999- 3299星尘 S9 Pro ——创新视觉盛宴：突破性 6.9英寸 1440 x 3088像素屏下摄像头设计，带来无界

## 🤼‍♀️ 3. 扩展阅读：Multi-Agent

上述讲解的是单一智能体的应用，如果想要构建Multi-Agent多智能体，可以阅读下面内容。

### Multi-Agent
用户可以引入多个扮演不同角色的LLM Agents参与到实际的任务中，Agents之间会进行竞争和协作等多种形式的动态交互，进而产生惊人的群体智能效果。复旦NLP团队在[Agent综述论文中-《The Rise and Potential of Large Language Model Based Agents: A Survey》](https://arxiv.org/pdf/2309.07864.pdf)提到，多代理系统（Multi-Agent System）是分布式人工智能领域中的热门研究问题之一，它主要关注代理们如何有效地协调并协作解决问题，多代理系统一般可以分成协作型互动、对抗型互动两种。

<div align="center">
<img src="https://img.alicdn.com/imgextra/i1/O1CN01WRSZhX1QJthdGpdWw_!!6000000001956-0-tps-1102-436.jpg" alt="image" style="width:750px;height:300px;">
</div>

**协作型互动**   
协作型Agent应用是实际应用中应用最广泛的模式，它可以有效提高任务效率，改善集体决策，解决单个智能体无法独立解决的复杂现实问题，最终实现协同互补的目标。协作型互动又分为无序合作与有序合作。
- 当所有代理自由地表达自己的观点、看法，以一种没有顺序的方式进行合作时，称为无序合作。
- 当所有代理遵循一定的规则，例如以流水线的形式逐一发表自己的观点时，整个合作过程井然有序，称为有序合作。
  
**对抗型互动**  
Agent应用之间以一种针锋相对（tit for tat）的方式进行互动。通过竞争、谈判、辩论的形式，代理抛弃原先可能错误的信念，对自己的行为或者推理过程进行有意义的反思，最终带来整个系统响应质量的提升。

#### Agent驱动的虚拟软件公司-ChatDev
ChatDev是全流程自动化软件开发框架，ChatDev可以比拟为一个由多智能体协作运营的虚拟软件公司，在“人类”用户指定一个具体的任务需求时，不同角色的智能体能够进行交互式协同，生产出一个完整软件（包括源代码、环境依赖说明书、用户手册等）。ChatDev的主要目标是提供一个基于大语言模型的易使用、高度定制化并且可扩展的框架，是研究群体智能的理想场景。   

ChatDev借鉴软件工程瀑布模型的思想，将其分为软件设计、系统开发、集成测试、文档编制四个主要环节。之后通过软件开发瀑布模型进一步拆解，形成由原子任务构成的交流链，整条链路可以视为软件生产线，链路中每个子任务通过充当专业角色（产品设计官、Python程序员、测试工程师）的智能体进行对话式信息交互和决策，驱动整条链路能够自主的进行需求分析、系统开发、集成测试、GUI构建、文档写作等全流程开发工作。   

<div align="center">
<img src="https://img.alicdn.com/imgextra/i4/O1CN01FucMo01slex37BpcP_!!6000000005807-0-tps-1220-1090.jpg" alt="image" style="width:500px;height:400px;">
</div>

驱动智能体交流对话的主要机制为：角色专业化（Role Specialization）、记忆流（Memory Stream）、自反思（Self-Reflection）:

- 角色专业化通过角色扮演机制（Role-Playing）确保每个智能体各司其职；每个角色都有其特定的目标和行为准则，这些准则指导智能体如何理解和回应用户的需求，帮助智能体在特定领域内提供更专业、更有针对性的服务。
- 记忆流通过将历史对话进行呈现，保证上下文感知的对话过程，捕捉用户在当前会话中提到的信息或者是记住用户的历史偏好和行为模式，并动态地对对话历史信息进行汇总和决策。
- 自反思机制在对话没有自动触发结束协议时生效，通过对自己行为和决策过程进行内部评估，分析智能体的响应是否达到了预期，通过自反思，智能体可以学习如何改进自己的行为，以更好地满足用户的需求。自反思还可以包括对交流策略的调整，以及对用户反馈的响应，从而使智能体能够不断进化和适应。

<div align="center">
<img src="https://img.alicdn.com/imgextra/i3/O1CN011P5glB1hCt6o6smB0_!!6000000004242-0-tps-1366-486.jpg" alt="image" style="width:550px;height:180px;">
</div>

#### 多模态大模型协作系统-HuggingGPT
 
目前大模型解决单一模态的问题已经涌现出很多方法了，但是还无法处理复杂的人工智能任务如多模态内容的生成。  

LLM具有强大的语言能力，HuggingFace具有丰富的人工智能模型，如果能够将这两种能力结合起来，那就能够处理复杂的任务了；浙江大学联合微软亚洲研究院提出了HuggingGPT，它能够覆盖不同模态和领域的众多复杂人工智能任务，在语言、视觉、语音和其他具有挑战性的任务方面取得令人印象深刻的成果，这为 AGI 铺平了新的道路。

<div align="center">
<img src="https://img.alicdn.com/imgextra/i3/O1CN01txRxZ01r8RQCEESKO_!!6000000005586-0-tps-1082-480.jpg" alt="image" style="width:600px;height:280px;">
</div>


HuggingGPT 的工作可以分为四个阶段：

- 任务规划：使用 LLM 分析用户请求，将其分解为多个子任务，规划任务顺序和依赖关系。

- 模型选择：对于子任务，LLM 将根据模型描述来调用 HuggingFace 上的专家模型。

- 任务执行：每个专家模型执行所分配的子任务，返回执行结果。

- 响应生成：最后由 LLM 集成所有专家模型的结果，并为用户生成答案。


## ✅ 本节小结
通过本小节内容的学习，你已经学会了如何使用如何构建插件来构建Agent应用了。下一节将学习如何使用RAGAS框架来评估RAG应用的质量。

## 🔥 课后小测验

【多选题】2.5.1. 以下哪些代码片段可以正确创建一个使用夸克搜索的助手？( )

A. dashscope.Assistants.create(model='qwen-max', tools=['quark_search'])

B. dashscope.Assistants.create(model='qwen-max', tools=[{'type': 'quark_search'}])

C. dashscope.Assistants.create(model='qwen-max', tools={'quark_search'})

D. dashscope.Assistants.create(model='qwen-max', tool='quark_search')

E. dashscope.Assistants.create(model='qwen-max', tools=[{'name': 'quark_search'}])

F. dashscope.Assistants.create(model='qwen-max', tools=[{'type': 'quark_search', 'params': {}}])

答案：B, F (虽然B是示例代码，F更完整，params 可为空对象，表示使用默认参数)

<br>

【多选题】2.5.2. 以下示例代码中，query 方法是如何处理用户查询的？（ ）
```python
def query(self, query:str):
    '''
    query: string, the query string to the assistant. e.g. Who is the Jack Chou ?
    '''
    message = dashscope.Messages.create(self.thread.id, content=query)
    message_run = dashscope.Runs.create(self.thread.id, assistant_id=self.assistant.id)
    run_status = dashscope.Runs.wait(message_run.id, thread_id=self.thread.id)
    if run_status.required_action:
        self._forward_and_submit_outputs(run_status)
        run_status = dashscope.Runs.wait(run_status.id, thread_id=self.thread.id)
        
    msgs = dashscope.Messages.list(self.thread.id)
    answer = json.loads(json.dumps(msgs, default=lambda o: o.__dict__))['data'][0]['content'][0]['text']['value']
    return answer

```


A. 创建一个新的消息，并将用户查询作为消息内容。

B. 启动一个新的运行实例，并关联到指定的 Assistant。

C. 等待运行实例完成，并获取结果。

D. 如果运行实例需要调用函数，则调用 _forward_and_submit_outputs 方法。

E. 直接返回大语言模型生成的文本，无需处理。

F. 从消息列表中提取最终的答案文本。

答案：A, B, C, D, F